In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
l =[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        l.append(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/padchestpa-1980/PadChestPA_1980.csv
/kaggle/input/padchestpa/PadChestPA.csv
/kaggle/input/translations/all_translations.txt


In [3]:
from transformers import pipeline
import os
import pandas as pd
from tqdm import tqdm
import torch

from transformers import pipeline
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset

2025-08-07 13:06:03.812621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754571964.114358      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754571964.200164      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
df = pd.read_csv(l[0])
len(df)

9496

In [ ]:

import re
def clean_segment(text):
    """
    Clean raw text by removing extra dots, spaces, and formatting issues
    """
    # Split into lines for processing
    lines = text.split('\n')
    cleaned_lines = []

    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue

        # Remove multiple consecutive spaces and replace with single space
        line = re.sub(r' +', ' ', line)

        # Remove multiple consecutive dots and replace with single dot
        line = re.sub(r'\.{2,}', '.', line)

        # Remove isolated dots (dots surrounded by spaces)
        line = re.sub(r'\s+\.\s+', ' ', line)

        # Remove dots at the beginning of lines
        line = re.sub(r'^\s*\.\s*', '', line)

        # Clean up any remaining multiple spaces
        line = re.sub(r' +', ' ', line)

        # Strip leading and trailing whitespace
        line = line.strip()

        # Add cleaned line if it's not empty
        if line:
            cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)



In [7]:
df_clean = df.dropna()
df_clean['Report'] = df_clean['Report'].apply(clean_segment)
reports = df_clean['Report'].to_list()
# dataset = Dataset.from_pandas(pd.DataFrame({"text": reports}))
len(df_clean)

/tmp/ipykernel_36/4211703445.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Report'] = df_clean['Report'].apply(clean_segment)


9490

In [6]:
from transformers import pipeline

# model_name = "facebook/nllb-200-distilled-600M"
model_name = "facebook/nllb-200-1.3B"

translator = pipeline(
    "translation", 
    model=model_name, 
    device=0,
    src_lang="spa_Latn",  # Spanish
    tgt_lang="eng_Latn"   # English
)


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
import os
from tqdm import tqdm

output_folder = "/kaggle/working"
os.makedirs(output_folder, exist_ok=True)

batch_size = 32
translation_count = 0

output_path = os.path.join(output_folder, "all_translations.txt") #pass it back 
print("Starting translation and saving...")

with open(output_path, "w", encoding="utf-8") as output_file:
    for idx in tqdm(range(0, len(reports), batch_size), desc="Translating and saving batches"):
        batch = reports[idx:idx + batch_size]
        translations_batch = translator(batch, max_length=256)
        
        for translated in translations_batch:
            output_file.write(translated['translation_text'] + "\n")
        output_file.flush()  # flush once per batch for speed
        translation_count += len(translations_batch)

print(f"Completed {translation_count} translations. File saved at: {output_path}")


Starting translation and saving...


Translating and saving batches: 100%|██████████| 297/297 [1:31:54<00:00, 18.57s/it]

Completed 9490 translations. File saved at: /kaggle/working/all_translations.txt


In [ ]:
def create_medical_prompt(medical_text):
    return f"""
You are a board-certified radiologist with expertise in reviewing and standardizing medical documentation. Your task is to revise the following chest X-ray report to improve its clarity, professional tone, and grammatical accuracy.

**Instructions:**
1.  Correct all grammatical errors and typos.
2.  Ensure the use of standard, concise medical terminology.
3.  Improve the overall structure and flow for better readability.
4.  Crucially, DO NOT add, invent, or speculate on any medical findings that are not explicitly mentioned in the original text. Your role is to refine the existing information, not to make a new diagnosis.
5.  Output ONLY the revised medical report text and nothing else.

**Original Report:**
"{medical_text}"

**Revised Report:**
"""

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login



MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

# File paths
INPUT_FILE = "/kaggle/input/translations/all_translations.txt" # file that was generatef earlier using NLLB
OUTPUT_FILE = "/kaggle/working/revised_reports.csv"

BATCH_SIZE = 1 if torch.cuda.is_available() else 1 
MAX_LENGTH = 512  


In [3]:


#Loading model:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16, 
    device_map="auto" if torch.cuda.is_available() else "cpu",
    low_cpu_mem_usage=True
)
        
print("Model loaded with float16 precision!")
     


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded with float16 precision!


In [ ]:

def process_batch(batch_texts):
    
    results = []
    
    for text in batch_texts:
        try:
            prompt = create_medical_prompt(text)
            
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_LENGTH,
                padding=True
            )
            
            # Move to device
            if torch.cuda.is_available():
                inputs = inputs.to("cuda")
            
            with torch.inference_mode():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=150,
                    temperature=0.3,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )
            
            
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            improved = generated[len(prompt):].strip()
            
            if improved.startswith("Improved report:") or improved.startswith("Report:"):
                improved = improved.split(":", 1)[1].strip()
            
            results.append(improved if improved else text)
            
        except Exception as e:
            print(f"Error processing text: {str(e)[:100]}...")
            results.append(text)  # Fallback to original
    
    return results




In [5]:
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
        
print(f"Found {len(lines)} medical texts")

Found 9490 medical texts


In [ ]:
import csv
import tqdm
import torch

processed_count = 0

with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as out_f:
    writer = csv.writer(out_f)
    
    writer.writerow(["Row", "Original Text", "Corrected Text"]) # return a csv file with these 3 columns for comparision
    
    for i in tqdm.tqdm(range(0, len(lines), BATCH_SIZE)):
        batch = lines[i:i + BATCH_SIZE]
        
        improved_texts = process_batch(batch)

        # Save results
        for original, improved in zip(batch, improved_texts):
            processed_count += 1
            writer.writerow([
                processed_count,
                original.strip(),
                improved.strip()
            ])

        # Clear GPU cache to save menory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

print("\nProcessing Complete!")
print(f"Successfully processed {processed_count} medical texts")
print(f"Results saved to: {OUTPUT_FILE}")


100%|██████████| 9490/9490 [4:46:06<00:00,  1.81s/it]   


Processing Complete!
Successfully processed 9490 medical texts
Results saved to: /kaggle/working/revised_reports.csv
